In [ ]:
import numpy as np
import pandas as pd

# Duomenys
data_raw = pd.DataFrame({
    'Lytis': [0, 1, 0, 1, None, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    'Ūgis': [165, 183, 171, 194, 158, 185, 160, 175, 169, 180, 164, 182, 167, 190, 163, 177],
    'Svoris': [55, 95, 60, 102, 54, 90, 52, 78, 63, 85, 52, 92, 58, 98, 54, 77]
})

# Normalizacija
def normalize(df):
    return (df - df.min()) / (df.max() - df.min())

features = normalize(data_raw[['Ūgis', 'Svoris']])
labels = data_raw['Lytis'].dropna().astype(float).values
#mokymo data be 5to paciento
train_data = features.drop(index=4).values  # be 5-o paciento

# Sigmoid funkcijos
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
#sigmoid isvestine reikalinga mokymui
def deriv_sigmoid(x):
    fx = sigmoid(x)
    return fx * (1 - fx)

#nuostolio funkcija - matuoja kiek tinklas klysta - naudojam gradienta (svoriams atnaujinti)
def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

# Neuroninis tinklas
class OurNeuralNetwork:
    def __init__(self):
        self.w1 = np.random.normal()
        self.w2 = np.random.normal()
        self.w3 = np.random.normal()
        self.w4 = np.random.normal()
        self.w5 = np.random.normal()
        self.w6 = np.random.normal()
        self.b1 = 1.0
        self.b2 = 1.0
        self.b3 = 1.0
#proganazavimo funkcija
    def feedforward(self, x):
        h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
        h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
        o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
        return o1
#mokymo funkcija
    def train(self, data, all_y_trues, learn_rate, epochs):
        for epoch in range(epochs):
            for x, y_true in zip(data, all_y_trues):
                sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
                h1 = sigmoid(sum_h1)

                sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
                h2 = sigmoid(sum_h2)

                sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
                o1 = sigmoid(sum_o1)
                y_pred = o1
                #nuostolio funkcijos isvestine-bendra klaida
                d_L_d_ypred = -2 * (y_true - y_pred)

                d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
                d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
                d_ypred_d_b3 = deriv_sigmoid(sum_o1)

                d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
                d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)

                d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
                d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
                d_h1_d_b1 = deriv_sigmoid(sum_h1)

                d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
                d_h2_d_w4 = x[1] * deriv_sigmoid(sum_h2)
                d_h2_d_b2 = deriv_sigmoid(sum_h2)
                #mokosi is klaidu ir gerina prognozes - Kiekvienas svoris ir baies (įtrauka) atnaujinami, kad tinklas mokytųsi ir mažintų klaida
                  self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
                  self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
                  self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

                  self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
                  self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
                  self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

                  self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
                  self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
                  self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

        y_preds = np.apply_along_axis(self.feedforward, 1, data)
        loss = mse_loss(all_y_trues, y_preds)
        return loss

# Bandymų parametrai
tests = [
    (0.01, 500),
    (0.01, 1000),
    (0.05, 500),
    (0.05, 1000),
    (0.1, 500),
    (0.1, 1000)
]

results = []

for lr, ep in tests:
    net = OurNeuralNetwork()
    loss = net.train(train_data, labels, learn_rate=lr, epochs=ep)
    predic+tion = net.feedforward(features.iloc[4].values)
    results.append({'Mokymosi koeficientas': lr, 'Epochų skaičius': ep, 'Nuostolis': round(loss, 4), 'Prognozė (Vardas 5)': round(prediction, 4)})

# Rezultatų lentelė
results_df = pd.DataFrame(results)
print(results_df)

prediction = net.feedforward(features.iloc[4].values)

# Lyties interpretacija
if prediction >= 0.5:
    print(f"Prognozuojama lytis: vyras (1), tikimybė: {prediction:.4f}")
else:
    print(f"Prognozuojama lytis: moteris (0), tikimybė: {prediction:.4f}")


   Mokymosi koeficientas  Epochų skaičius  Nuostolis  Prognozė (Vardas 5)
0                   0.01              500     0.1299               0.3342
1                   0.01             1000     0.0448               0.1292
2                   0.05              500     0.0159               0.0549
3                   0.05             1000     0.0056               0.0384
4                   0.10              500     0.0059               0.0333
5                   0.10             1000     0.0017               0.0060
Prognozuojama lytis: moteris (0), tikimybė: 0.0060
